# **Applicazione**

In [ ]:
%pip install pandas
%pip install openpyxl
%pip install tqdm
%pip install requests
%pip install PyQt5
%pip install beautifulsoup4
%pip install sklearn
%pip install matplotlib
%pip install seaborn


In [ ]:
import sys
import pandas as pd
import requests
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QPushButton, QLabel,
                             QFileDialog, QMessageBox, QProgressBar)
from PyQt5.QtGui import QFont, QColor, QPalette
from PyQt5.QtCore import Qt
from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# Inizializza il traduttore di Hugging Face per la traduzione in inglese
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Funzione per pulire il dataset CRM Input
def clean_data(df, progress_bar):
    df = df.drop(columns=["Contact Name", "Contact Job Title"])
    df = df.drop_duplicates().dropna()
    progress_bar.setValue(10)
    tqdm.pandas(desc="Extracting domains")
    df['Email Domain'] = df['Contact E-mail'].progress_apply(lambda email: email.split('@')[-1])

    public_domains = ['gmail.com', 'hotmail.com', 'yahoo.com', 'outlook.com', 'live.com', 'icloud.com', 'unknown.com']
    df = df[~df['Email Domain'].isin(public_domains)]
    progress_bar.setValue(30)
    
    def check_domain_exists(domain):
        try:
            response = requests.get(f"http://{domain}", timeout=2)
            return response.status_code == 200
        except requests.RequestException:
            return False

    tqdm.pandas(desc="Verifying domains")
    df['Domain Exists'] = df['Email Domain'].progress_apply(check_domain_exists)
    df = df[df['Domain Exists']].drop(columns=['Domain Exists'])

    df['Main Domain Name'] = df['Email Domain'].apply(lambda domain: domain.split('.')[0])
    df = df.drop_duplicates(subset=["Main Domain Name"]).drop(columns=['Main Domain Name'])
    df = df.drop_duplicates(subset=["Contact E-mail"]).drop_duplicates(subset=["Email Domain"])
    df = df.drop_duplicates()
    df = df.dropna()
    progress_bar.setValue(50)
    return df

# Funzione per pulire il dataset delle categorie
def clean_categories(df):
    return df.drop_duplicates().dropna().drop(columns=['Can they buy the solution?', 'Can they influence the buying decision?'])

# Funzione per estrarre informazioni e tradurre
def extract_meta_information(soup):
    info = {}
    description = soup.find('meta', attrs={'name': 'description'})
    if description:
        info['meta_description'] = description.get('content', '')
    og_description = soup.find('meta', attrs={'property': 'og:description'})
    if og_description:
        info['og_description'] = og_description.get('content', '')
    keywords = soup.find('meta', attrs={'name': 'keywords'})
    if keywords:
        info['meta_keywords'] = keywords.get('content', '')

    paragraphs = soup.find_all('p')
    if paragraphs:
        paragraph_text = " ".join([p.get_text().strip() for p in paragraphs[:7] if p.get_text().strip()])
        info['paragraphs'] = paragraph_text
    return info

# Funzione per estrarre e tradurre le informazioni in inglese
def extract_and_translate_company_info(url, progress_bar, index, total):
    try:
        response = requests.get(url, timeout=2)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_info = extract_meta_information(soup)
            description_parts = [meta_info.get(key, '') for key in ['meta_description', 'og_description', 'paragraphs']]
            full_description = " ".join(list(dict.fromkeys(description_parts)))[:700]
            translated_description = translator(full_description, max_length=700)[0]['translation_text']
            progress_bar.setValue(50 + int(30 * (index + 1) / total))
            return translated_description
        return "Company information not available."
    except:
        return "Company information not available."

# Funzione per aggiungere la colonna "Description" con traduzione
def add_description_column(df, progress_bar):
    descriptions = []
    for i, domain in enumerate(tqdm(df['Email Domain'], desc="Extracting and translating company information", unit="company")):
        website_url = f"http://{domain}"
        descriptions.append(extract_and_translate_company_info(website_url, progress_bar, i, len(df)))
        progress_bar.setValue(50 + int(30 * (i + 1) / len(df)))
    df['Description'] = descriptions
    return df

# Funzione per classificare basata su TF-IDF e similarità coseno
def classify_with_cosine_similarity(df_data, df_context, progress_bar):
    all_texts = df_data['Description'].tolist() + df_context['Notes'].tolist()
    vectorizer = TfidfVectorizer().fit(all_texts)
    company_vectors = vectorizer.transform(df_data['Description'])
    category_vectors = vectorizer.transform(df_context['Notes'])

    best_matches = []
    for i, company_vector in enumerate(tqdm(company_vectors, desc="Classifying", unit="company")):
        similarities = cosine_similarity(company_vector, category_vectors)
        best_match_idx = similarities.argmax()
        best_category = df_context.iloc[best_match_idx]['Player']
        best_matches.append(best_category)
        progress_bar.setValue(80 + int(20 * (i + 1) / len(df_data)))

    df_data['Best Category'] = best_matches
    return df_data

# Funzione per aggiungere la colonna "Is Target"
def add_target_column(df_data, df_context):
    target_map = df_context.set_index('Player')['Is Target'].to_dict()
    df_data['Is Target'] = df_data['Best Category'].apply(lambda category: target_map.get(category, "No"))
    return df_data

# Classe dell’applicazione
class ExcelApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Tema scuro futuristico
        palette = QPalette()
        palette.setColor(QPalette.Window, QColor(30, 30, 30))
        palette.setColor(QPalette.WindowText, QColor(255, 255, 255))
        self.setPalette(palette)
        
        title = QLabel("CRM Classifier", self)
        title.setFont(QFont("Arial", 16, QFont.Bold))
        layout.addWidget(title, alignment=Qt.AlignCenter)

        # Ingrandisci la descrizione e limita la larghezza al 70%
        description = QLabel("Tool to classify companies in CRM by comparing descriptions with context categories.", self)
        description.setWordWrap(True)
        description.setAlignment(Qt.AlignCenter)
        description.setMaximumWidth(int(self.width() * 0.7))  # Limita la larghezza
        layout.addWidget(description, alignment=Qt.AlignCenter)

        # Etichetta per CRM Input File
        self.crm_info_label = QLabel("The file must be an Excel (.xlsx) with a 'Contact E-mail' column.", self)
        layout.addWidget(self.crm_info_label, alignment=Qt.AlignCenter)

        # Caricamento file CRM Input
        self.upload_data_btn = QPushButton('Upload CRM Input File')
        self.upload_data_btn.clicked.connect(self.load_data_file)
        layout.addWidget(self.upload_data_btn, alignment=Qt.AlignCenter)

        # Etichetta per Context File
        self.context_info_label = QLabel("The file must be an Excel (.xlsx) with 'Player' and 'Notes' columns.", self)
        layout.addWidget(self.context_info_label, alignment=Qt.AlignCenter)

        # Caricamento file Context
        self.upload_context_btn = QPushButton('Upload Context File')
        self.upload_context_btn.clicked.connect(self.load_context_file)
        layout.addWidget(self.upload_context_btn, alignment=Qt.AlignCenter)

        # Barra di avanzamento
        self.progress_bar = QProgressBar(self)
        layout.addWidget(self.progress_bar)

        # Pulsante di analisi
        self.analyze_btn = QPushButton('Run Analysis')
        self.analyze_btn.clicked.connect(self.run_analysis)
        self.analyze_btn.setEnabled(False)
        layout.addWidget(self.analyze_btn, alignment=Qt.AlignCenter)

        # Pulsante per scaricare il file classificato
        self.download_btn = QPushButton('Download Classified File')
        self.download_btn.clicked.connect(self.download_file)
        self.download_btn.setEnabled(False)
        layout.addWidget(self.download_btn, alignment=Qt.AlignCenter)

        self.setLayout(layout)
        self.setWindowTitle('CRM Classifier')
        self.setGeometry(300, 300, 500, 300)
        self.show()

    def load_data_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Select CRM Input File", "", "Excel Files (*.xlsx)")
        if file_path:
            self.df_data = pd.read_excel(file_path)
            self.upload_data_btn.setText(f'Loaded: {file_path}')
            if hasattr(self, 'df_context'):
                self.analyze_btn.setEnabled(True)

    def load_context_file(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Select Context File", "", "Excel Files (*.xlsx)")
        if file_path:
            self.df_context = pd.read_excel(file_path)
            self.upload_context_btn.setText(f'Loaded: {file_path}')
            if hasattr(self, 'df_data'):
                self.analyze_btn.setEnabled(True)

    def run_analysis(self):
        try:
            self.progress_bar.setValue(0)
            self.df_data_cleaned = clean_data(self.df_data, self.progress_bar)
            self.df_context_cleaned = clean_categories(self.df_context)
            self.df_data_with_description = add_description_column(self.df_data_cleaned, self.progress_bar)

            # Classifica utilizzando la similarità coseno avanzata
            self.df_data_classified = classify_with_cosine_similarity(
                self.df_data_with_description, self.df_context_cleaned, self.progress_bar
            )

            # Aggiungi colonna "Is Target"
            self.df_data_classified = add_target_column(self.df_data_classified, self.df_context_cleaned)
            
            # Abilita pulsante di download
            self.download_btn.setEnabled(True)
            QMessageBox.information(self, 'Analysis Complete', 'Analysis successfully completed!')

        except Exception as e:
            QMessageBox.critical(self, 'Error', f'An error occurred during analysis: {str(e)}')
        finally:
            self.progress_bar.setValue(100)  # Set progress bar to 100% when finished

    def download_file(self):
        try:
            output_path, _ = QFileDialog.getSaveFileName(self, "Save Classified File", "", "Excel Files (*.xlsx)")
            if output_path:
                self.df_data_classified.to_excel(output_path, index=False)
                QMessageBox.information(self, 'Success', 'File saved successfully.')
                
                # Chiudi l'applicazione dopo aver salvato il file
                QApplication.quit()
                
            else:
                QMessageBox.warning(self, 'Warning', 'File not saved.')
        except Exception as e:
            QMessageBox.critical(self, 'Error', f'Error saving the file: {str(e)}')

if __name__ == '__main__':
    app = QApplication(sys.argv)
    app.setStyle('Fusion')  # Apply futuristic dark theme
    dark_palette = QPalette()
    dark_palette.setColor(QPalette.Window, QColor(45, 45, 45))
    dark_palette.setColor(QPalette.WindowText, QColor(208, 208, 208))
    dark_palette.setColor(QPalette.Base, QColor(30, 30, 30))
    dark_palette.setColor(QPalette.AlternateBase, QColor(45, 45, 45))
    dark_palette.setColor(QPalette.ToolTipBase, QColor(208, 208, 208))
    dark_palette.setColor(QPalette.ToolTipText, QColor(208, 208, 208))
    dark_palette.setColor(QPalette.Text, QColor(208, 208, 208))
    dark_palette.setColor(QPalette.Button, QColor(53, 53, 53))
    dark_palette.setColor(QPalette.ButtonText, QColor(208, 208, 208))
    dark_palette.setColor(QPalette.Highlight, QColor(142, 45, 197).darker())
    dark_palette.setColor(QPalette.HighlightedText, QColor(208, 208, 208))
    app.setPalette(dark_palette)

    ex = ExcelApp()
    sys.exit(app.exec_())

           
